# Steamspy Data Cleaning

*This forms part of a larger series of posts for my [blog](http://nik-davis.github.io) on downloading, processing and analysing data from the steam store. [See all posts here](http://nik-davis.github.io/tags/steam).*

In [1]:
# view software version information

# http://raw.github.com/jrjohansson/version_information/master/version_information.py
%load_ext version_information
%reload_ext version_information

%version_information numpy, pandas

Software versions
Python 3.7.3 64bit [MSC v.1900 64 bit (AMD64)]
IPython 7.5.0
OS Windows 10 10.0.17763 SP0
numpy 1.16.3
pandas 0.24.2
Thu May 30 18:10:37 2019 GMT Summer Time

In [3]:
# standard library imports
from ast import literal_eval
import itertools
import time
import re

# third-party imports
import numpy as np
import pandas as pd

# customisations
pd.set_option("max_columns", 100)

In [4]:
raw_steamspy_data = pd.read_csv('../data/raw/steamspy_data.csv')
raw_steamspy_data.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,124534,3339,0,"10,000,000 .. 20,000,000",17612,709,317,26,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,14923,"{'Action': 2681, 'FPS': 2048, 'Multiplayer': 1..."
1,20,Team Fortress Classic,Valve,Valve,NaN,3318,633,0,"5,000,000 .. 10,000,000",277,15,62,15,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,87,"{'Action': 208, 'FPS': 188, 'Multiplayer': 172..."
2,30,Day of Defeat,Valve,Valve,NaN,3416,398,0,"5,000,000 .. 10,000,000",187,0,34,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Spain",Action,130,"{'FPS': 138, 'World War II': 122, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,1273,267,0,"5,000,000 .. 10,000,000",258,0,184,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,4,"{'Action': 85, 'FPS': 71, 'Multiplayer': 58, '..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,5250,288,0,"5,000,000 .. 10,000,000",624,0,415,0,499.0,499.0,0.0,"English, French, German, Korean",Action,71,"{'FPS': 235, 'Action': 211, 'Sci-fi': 166, 'Si..."


In [6]:
raw_steamspy_data.isnull().sum()

appid                  0
name                   5
developer            197
publisher            280
score_rank         29177
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                 29
initialprice          22
discount              22
languages             94
genre                152
ccu                    0
tags                   0
dtype: int64

In [10]:
for col in raw_steamspy_data.columns:
    print('\n\n', col, '\n\n')
    display(raw_steamspy_data[raw_steamspy_data[col].isnull()].head())



 appid 




,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags




 name 




,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
481,17760,NaN,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
7235,396420,NaN,NaN,NaN,NaN,22,9,0,"20,000 .. 50,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,"{'Free to Play': 24, 'Action': 21, 'Indie': 21..."
9553,460250,NaN,Jeroen Wimmers,Jeroen Wimmers,NaN,44,4,0,"0 .. 20,000",0,0,0,0,899.0,899.0,0.0,"English, French, Italian, German, Spanish - Sp...","Casual, Indie",0,"{'Indie': 34, 'Casual': 24, 'Puzzle': 15, 'Min..."
22244,806160,NaN,Paleno Games,Paleno Games,NaN,4,13,0,"0 .. 20,000",0,0,0,0,99.0,99.0,0.0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Casual, Indie",0,"{'Action': 22, 'Indie': 22, 'Casual': 21, 'Adv..."
27324,965340,NaN,2nd Studio,2nd Studio,NaN,31,1,0,"0 .. 20,000",0,0,0,0,199.0,199.0,0.0,"English, German, Danish, Japanese, Russian, Si...","Action, Indie, Simulation",0,"{'Indie': 32, 'Sexual Content': 31, 'Action': ..."




 developer 




...

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
26,852,ValveTestApp852,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
147,4330,Star Trek: DAC - Demo,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
256,8740,Puzzlegeddon,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,1,[]
264,8955,Borderlands DLC: Claptrap's New Robot Revolution,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
285,9730,Tycoon City: New York,NaN,Retroism,NaN,58,102,0,"20,000 .. 50,000",0,0,0,0,999.0,999.0,0.0,English,NaN,0,"{'Simulation': 34, 'Management': 18, 'City Bui..."




 ccu 




,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags




 tags 




,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags


In [11]:
raw_steamspy_data['owners'].value_counts()

0 .. 20,000                   20234
20,000 .. 50,000               3218
50,000 .. 100,000              1785
100,000 .. 200,000             1471
200,000 .. 500,000             1373
500,000 .. 1,000,000            556
1,000,000 .. 2,000,000          311
2,000,000 .. 5,000,000          210
5,000,000 .. 10,000,000          49
10,000,000 .. 20,000,000         22
20,000,000 .. 50,000,000          3
50,000,000 .. 100,000,000         2
100,000,000 .. 200,000,000        1
Name: owners, dtype: int64

In [25]:
i = 0
while i < 10:
    print(10**i, end=', ')
    i += 1
    
ml = [10**i for i in range(8)]
ml

1, 10, 100, 1000, 10000, 100000, 1000000, 10000000, 100000000, 1000000000, 

[1, 10, 100, 1000, 10000, 100000, 1000000, 10000000]

In [24]:
raw_steamspy_data['positive'].max()

2644404

In [27]:
pd.cut(raw_steamspy_data['positive'], bins=[10**i for i in range(8)]).value_counts()

(10, 100]              11031
(1, 10]                 7345
(100, 1000]             5567
(1000, 10000]           1904
(10000, 100000]          402
(100000, 1000000]         33
(1000000, 10000000]        1
Name: positive, dtype: int64

In [28]:
pd.cut(raw_steamspy_data['negative'], bins=[10**i for i in range(8)]).value_counts()

(1, 10]                9205
(10, 100]              9094
(100, 1000]            3506
(1000, 10000]           753
(10000, 100000]          60
(100000, 1000000]         4
(1000000, 10000000]       0
Name: negative, dtype: int64

In [60]:
pos = pd.cut(raw_steamspy_data['positive'], bins=[10**i for i in range(8)])
neg = pd.cut(raw_steamspy_data['negative'], bins=[10**i for i in range(8)])

div = raw_steamspy_data['positive'] / raw_steamspy_data['negative']
pd.cut(div, bins=[0, 1, 5, 10, 100]).value_counts()

(1, 5]       12740
(0, 1]        4844
(5, 10]       4105
(10, 100]     3108
dtype: int64